In [1]:
import os
import warnings

import pandas as pd
import torch

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, 
    DataCollatorForSeq2Seq, 
)

from datasets import Dataset

import nltk

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jake/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(1, 16)

# Paths and Names

In [4]:
### paths and names

DATA_PATH = 'data/preprocess_v2.pickle'
MODEL_CHECKPOINT = '.log/ainize_kobart_news_v2/checkpoint-3170'

# Training Args

In [14]:
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 1

predict_with_generate=True
generation_max_length=128

# Model & Tokenizer & Metric

- 모델과 토크나이저, 그리고 평가지표를 계산하는 데 사용할 함수를 불러옵니다.
- 모델의 config에는 사용하지 않는 설정이 포함되어 있습니다. 삭제합니다.

In [5]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# Functions

- 모델 훈련을 위해 전처리를 수행해주는 함수를 선언합니다.
- 입력 문장이 input이 되고 요약문이 label이 됩니다.
- tokenizer를 사용해 input_ids로 변환하고 입력값에는 attention_mask를 생성해줍니다.
- 입력 문장에 prefix를 추가하여 성능 실험을 해볼 수 있으나 지금은 적용하지 않겠습니다.

In [7]:
prefix = "generate keyphrases: "

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Inputs and Labels

In [8]:
data_df = pd.read_pickle(DATA_PATH)

In [9]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.2, seed=100)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [10]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

Map (num_proc=16):   0%|          | 0/1268 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/317 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1268
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 317
})


In [11]:
tokenizer.decode(train_dataset['input_ids'][0])

'<s>generate keyphrases: 광주시, 물류운송 드론 기술개발 공모 선정 광주시가 2백킬로그램급 화물운송용 드론 기술개발 공모사업에 선정됐습니다. 이번 사업은 오는 2025년까지 수소연료전지를 기반으로 최대 시속 백킬로미터의 속도로 탑재 중량 2백 킬로그램급 드론개발과 실증 사업 기반을 구축하는 것입니다. 광주시는 지난 1월 LIG넥스원 등과 화물용 드론 개발 업무협약을 체결했으며, 수소연료전지 기술 개발과 함께 물류 운송용 드론 시장 선점을 통한 지역 산업 성장에도 도움이 될 것으로 크게 기대하고 있습니다. 최송현</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [12]:
tokenizer.decode(train_dataset['labels'][0])

'<s>광주시,물류운송 드론,기술개발 공모,선정,2백킬로그램급 화물운송용 드론,2025년,수소연료전지</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Train

- 상단에서 선언한 HP와 각종 변수들로 training_args를 생성합니다.

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./.temp',
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
)

- 배치 단위로 padding처리를 하여 훈련 속도를 높히기 위해 data_collator를 선언합니다.
- 생성한 객체들을 trainer에 포함시켜 훈련 준비를 마칩니다.

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,
)

- 훈련을 시작합니다.

In [ ]:
# trainer.train()

- 옵티마이저 스테이트를 포함한 훈련 재개에 사용할 파일들을 삭제해줍니다.
- 추후 불러올 모델과 토크나이저 관련 파일만 남겨둡니다.

In [ ]:
# keep = [
#     'added_tokens.json',
#     'config.json',
#     'pytorch_model.bin',
#     'special_tokens_map.json',
#     'tokenizer.json',
#     'tokenizer_config.json',
#     'vocab.txt'
# ]

# ckpts = os.listdir(output_dir)
# for ckpt in ckpts:
#     ckpt = os.path.join(output_dir, ckpt)
#     for item in os.listdir(ckpt):
#         if item not in keep:
#             os.remove(os.path.join(ckpt, item))

# Generate

- 훈련을 마치면 Evaluation Loss 기준 Best 모델이 로드되어 있습니다.
- trainer는 Greedy Search를 수행하도록 설정되어 있습니다.
- Evaluation 데이터를 활용해 모델의 출력을 간단히 살펴봅니다.

In [19]:
preds = trainer.predict(eval_dataset)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [20]:
for data, pred in zip(eval_dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')

정답: 지한솔,KLPGA 투어 E1 채리티오픈
예측: KLPGA 투어 E1 채리티오픈,하민송,지한솔,2R 선두,하민송

정답: 박명수,한수민,딸,데이트,사춘기,민서,SNS
예측: 박박수,한수민,딸,데이트,사춘기,예원학교 한국무용과

정답: 크리스찬 루부탱,더보이즈 영훈
예측: 더보이즈 영훈,더비들,더보이즈 영훈,팝업스토어,포토콜

정답: 롯데 자이언츠,5연패 탈출,KBO리그,KIA 타이거즈,17-9 승리,안치홍,딕슨 마차도
예측: 롯데,KBO리그,17-9,17-9,롯데 자이언츠,2021 KBO리그 홈경기,KBO리그

정답: 프랑스 축구,선두 파리 생제르맹,복통,콘테 감독,담낭염
예측: 파SG,음바페 실축,프랑스 축구,파리 생제르맹,승리

정답: 김치 발효,유산균 발효,발효과학,풀무원,발효 김치,유산균,류코노스톡
예측: 풀무원,발효과학,풀무원 톡톡김치,전 제품 리뉴얼,한국 김치만의 독창적인 특징인 '발효 풍미',한국인이 가장 좋아하는 '겨울 김장김치의 시원한 발효 감칠맛',발효과학,풀무원 톡톡김치

정답: 카카오,태·조·이·방·원,코스피지수,코스닥지수,인터넷,삼전은 '5만전자',신고가
예측: 코스전은 '5만전자',태·조·이·방·원

정답: 이별 리콜,성유리 외모 지적
예측: 외별 지적,성유리,이별 리콜,연인,사랑,이별

정답: 남다름,박서준,이른 나이에 자진 입대,정해인,유승호
예측: 박서준,남다름,이른 나이에 자진 입대,이승호,유승호

정답: 신세계면세점 부산점,소외계층 청소년 장학금,장학금 지원
예측: 신세계면세점 부산점,소외계층 청소년 장학금,장학금 지원,부산공동모금회,부산시청,부산점 점장,부산 지역민

정답: 전유진 달성군수 후보 선거사무소 개소
예측: 전유진,대구 달성군수 후보,선거사무소,개소식,인사말,대구경북의 유일한 청년여성 후보,청년여성 후보

정답: 전용면적 84m2,첫 '20억 아파트',경기도
예측: 경기도 첫 '20억 아파트',전용면적 84m2 아파트,20억 클럽,공공 주도 공급,비현실적인 '공공 주도 공급',풍선효과

정답: K리그1,K리그